#####  create fact and dimesion tables and keep it in gold layer in delta format

In [0]:
from delta.tables import DeltaTable
from pyspark.sql.functions import *

In [0]:
sliver_source="abfss://silver@netflixmainstorage.dfs.core.windows.net"

In [0]:
df = spark.read.format("delta").load(sliver_source+"/titles")
df.createOrReplaceTempView("titles")

In [0]:
df = spark.read.format("delta").load(sliver_source+"/directors")
df.createOrReplaceTempView("directors")

In [0]:
df = spark.read.format("delta").load(sliver_source+"/category")
df.createOrReplaceTempView("category")

In [0]:
df = spark.read.format("delta").load(sliver_source+"/countries")
df.createOrReplaceTempView("countries")

In [0]:
df = spark.read.format("delta").load(sliver_source+"/cast")
df.createOrReplaceTempView("cast")

In [0]:
%sql
select * from countries

In [0]:
%sql
CREATE OR REPLACE TABLE netflix_catalog.net_schema.fact_title_master
USING DELTA
LOCATION 'abfss://gold@netflixmainstorage.dfs.core.windows.net/master_table'
AS
SELECT 
  try_cast(t.show_id AS INT) AS show_id,
  t.title,
  t.type,
  t.release_year,
  t.rating,
  d.director,
  c.cast,
  cat.listed_in AS category,
  ctr.country
FROM titles AS t
LEFT JOIN directors AS d ON try_cast(t.show_id AS INT)  = d.show_id 
LEFT JOIN cast AS c ON try_cast(t.show_id AS INT) = c.show_id
LEFT JOIN category AS cat ON try_cast(t.show_id AS INT)  = cat.show_id
LEFT JOIN countries AS ctr ON try_cast(t.show_id AS INT)  = ctr.show_id ;
